In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
pd.set_option('display.max_columns', None)

In [5]:
file_path = '../../data/processed/text_data.csv'

In [8]:
data = pd.read_csv(file_path)

In [9]:
data

,tour_name,tour_option_name,description,travelInclusions,diningInclusions,day_1_title,day_2_title,day_3_title,day_4_title,day_5_title,day_6_title,day_7_title,day_8_title,day_9_title,day_10_title,day_11_title,day_12_title,day_13_title,day_14_title,day_15_title,day_16_title,day_17_title,day_18_title,day_19_title,day_20_title,day_21_title,day_22_title,day_23_title,day_24_title,day_25_title,day_26_title,day_27_title,day_28_title,day_1_description,day_2_description,day_3_description,day_4_description,day_5_description,day_6_description,day_7_description,day_8_description,day_9_description,day_10_description,day_11_description,day_12_description,day_13_description,day_14_description,day_15_description,day_16_description,day_17_description,day_18_description,day_19_description,day_20_description,day_21_description,day_22_description,day_23_description,day_24_description,day_25_description,day_26_description,day_27_description,day_28_description
0,Majestic Alaska,Majestic Alaska,A wild escape brings your dream of Alaska to l...,"Must-see Highlights\n- Explore Valdez, Seward,...",Whats Included\n- An expert Travel Director an...,Start Your Adventure In Anchorage,Step Into The Wilderness Of Denali National Park,Uncover The Natural History Of Denali,Explore Denali And Venture To Fairbanks,Dig Deep Into Alaskan Culture Dive Into Cultur...,Take A Scenic Journey To Valdez,Spot The Wildlife Of Valdez,Travel Back To Anchorage Make Travel Matter,Glance At Glaciers Make Travel Matter,Depart Anchorage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Welcome to the edge of the wilderness – to a c...,Kickstart your day with a brief Anchorage City...,With the Denali National Park and Preserve jus...,Tune into your beautiful surrounds as you’ll h...,"After breakfast, you will enjoy a city tour of...",We journey south to the port community of Vald...,"Delve into the heritage and culture of Valdez,...",We begin our return journey to Anchorage this ...,We journey along the Scenic Seward Byway this ...,Our adventure through the rugged Alaskan wilde...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,South America Landscapes,South America Landscapes with Brazilian Amazon,You’ll discover why so many fall in love with ...,Sightseeing Highlights\n- Discover how locals ...,Dining Highlights\n- 1 Be My Guest\n- 1 Farewe...,Welcome to Santiago,Journey to Santiago,Onwards to Puerto Varas,Cross the Andes and Continue to Bariloche,Admire Bariloche,"Venture to Buenos Aires, the Paris of South Am...",Explore Buenos Aires,Buenos Aires Your Way,Discover Iguassu Falls,Soak in the Brazilian Side of Iguassu Falls,Off to Rio de Janeiro,Enjoy Rio de Janeiro and Sugarloaf Mountain,Uncover the Sights and Sounds of Rio de Janeiro,The Glamour of Manaus,The River of Manaus,The Diverse Fauna of the Amazon,Depart Manaus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Connect with the cosmopolitan verve of the Chi...,This morning’s city tour with a Local Speciali...,We fly south to the coastal city of Puerto Mon...,Look forward to a day of pinch me moments as y...,Soak up the sights and sounds of the 'Switzerl...,Leaving spectacular Patagonian landscapes behi...,Immerse yourself in the seamless mix of Europe...,"Get a taste of life as a porteño this morning,...",Journey to the thundering cascade of Iguassu F...,Rediscover the 'Big Waters' and return to Igua...,We bid farewell to the falls and fly to sultry...,Begin your exploration of the cidade maravilho...,Board a classic cogwheel train for the journey...,Fit in some last-minute exploring and souvenir...,We journey upstream today to see first-hand th...,Walk beneath the towering canopy of the world’...,"Visit Monkey Forest, a rehabilitation centre a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Highlights of Peru,Highlights of Peru,You’re in for an adventure on this tour of Per...,Iconic Experience\n- Lima: Take in the highlig...,Whats Included\n- An expert Travel Director an...,A

In [12]:
data.isna().sum()

tour_name               0
tour_option_name        0
description             0
travelInclusions      209
diningInclusions      310
                     ... 
day_24_description    760
day_25_description    765
day_26_description    766
day_27_description    771
day_28_description    772
Length: 61, dtype: int64

In [14]:
# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [15]:
# Load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()  # Set the model to evaluation mode to make prediction on the text

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [20]:
# Tokenizing texts in 'description' column
# input_ids, attention_masks = tokenize_texts(data['description'].fillna(''), max_len=512)

In [21]:
def tokenize_texts(texts, max_len=512):
    input_ids = []
    attention_masks = []

    for text in texts:
        # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
        input_id = tokenizer.encode(text, add_special_tokens=True, max_length=max_len, truncation=True)
        
        # Create attention mask.
        attention_mask = [1] * len(input_id)
        
        # Pad and create attention masks.
        # If `max_len` > len(input_id), pad the list of input ID's with the special token ID, 0.
        # And create corresponding attention masks with 0.
        padding_length = max_len - len(input_id)
        input_id = input_id + ([0] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)

        # Append to the lists.
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        
    return input_ids, attention_masks


In [26]:
import logging
logging.basicConfig(level=logging.INFO)

In [41]:
embeddings_dict = {}

batch_size = 8
output_dir = "embeddings"

for col in tqdm(data.columns, desc='Processing columns', unit='column'):
    print(f"Processing column: {col}")
    
    # Handle NaN and tokenize texts in the column
    texts = data[col].fillna('')
    input_ids, attention_masks = tokenize_texts(texts, max_len=512)  # Define the tokenize_texts function
    
    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    
    # Create DataLoader
    dataset = TensorDataset(input_ids, attention_masks)
    dataloader = DataLoader(dataset, batch_size=batch_size)

    # Set the model to evaluation mode and get embeddings
    model.eval()
    embeddings_list = []  # List to hold the embeddings of each batch
    
    for batch in tqdm(dataloader, desc=f'Processing batches in {col}', unit='batch'):
        input_ids_batch, attention_masks_batch = batch
        with torch.no_grad():
            outputs = model(input_ids=input_ids_batch, attention_mask=attention_masks_batch)
        
        # Extract embeddings from the last layer and append to the list
        embeddings_batch = outputs.last_hidden_state[:, 0, :].numpy()
        embeddings_list.append(embeddings_batch)
    
    # Concatenate embeddings from all batches and store them in the dictionary
    embeddings = np.concatenate(embeddings_list, axis=0)
    embeddings_dict[col] = embeddings
    
    # Write embeddings to a file
    output_file_path = os.path.join(output_dir, f"{col}_embeddings.npy")
    np.save(output_file_path, embeddings)

Processing columns:   0%|                            | 0/61 [00:00<?, ?column/s]

Processing column: tour_name



Processing batches in tour_name:  99%|██████▉| 96/97 [11:39<00:07,  7.86s/batch]


Processing columns:   2%|▎               | 1/61 [11:45<11:45:05, 705.08s/column]

Processing column: tour_option_name


...


Processing batches in day_27_description:   0%|       | 0/97 [00:00<?, ?batch/s]
Processing batches in day_27_description:   1%| | 1/97 [00:07<12:11,  7.62s/batc
Processing batches in day_27_description:   2%| | 2/97 [00:15<12:27,  7.87s/batc
Processing batches in day_27_description:   3%| | 3/97 [00:23<12:02,  7.68s/batc
Processing batches in day_27_description:   4%| | 4/97 [00:30<11:50,  7.64s/batc
Processing batches in day_27_description:   5%| | 5/97 [00:38<11:38,  7.59s/batc
Processing batches in day_27_description:   6%| | 6/97 [00:46<11:40,  7.69s/batc
Processing batches in day_27_description:   7%| | 7/97 [00:53<11:24,  7.61s/batc
Processing batches in day_27_description:   8%| | 8/97 [01:01<11:15,  7.59s/batc
Processing batches in day_27_description:   9%| | 9/97 [01:08<11:01,  7.52s/batc
Processing batches in day_27_description:  10%| | 10/97 [01:15<10:52,  7.50s/bat
Processing batches in day_27_description:  11%| | 11/97 [01:23<10:44,  7.50s/bat
Processing batches in day_2

Processing batches in day_27_description: 100%|█| 97/97 [12:09<00:00,  7.52s/bat
Processing columns:  98%|██████████████▊| 60/61 [12:49:29<12:13, 733.71s/column]

Processing column: day_28_description



Processing batches in day_28_description:   0%|       | 0/97 [00:00<?, ?batch/s]
Processing batches in day_28_description:   1%| | 1/97 [00:07<12:30,  7.81s/batc
Processing batches in day_28_description:   2%| | 2/97 [00:15<12:11,  7.70s/batc
Processing batches in day_28_description:   3%| | 3/97 [00:22<11:52,  7.58s/batc
Processing batches in day_28_description:   4%| | 4/97 [00:30<11:35,  7.48s/batc
Processing batches in day_28_description:   5%| | 5/97 [00:37<11:32,  7.53s/batc
Processing batches in day_28_description:   6%| | 6/97 [00:45<11:20,  7.48s/batc
Processing batches in day_28_description:   7%| | 7/97 [00:52<11:14,  7.50s/batc
Processing batches in day_28_description:   8%| | 8/97 [01:00<11:01,  7.44s/batc
Processing batches in day_28_description:   9%| | 9/97 [01:07<10:53,  7.43s/batc
Processing batches in day_28_description:  10%| | 10/97 [01:14<10:45,  7.42s/bat
Processing batches in day_28_description:  11%| | 11/97 [01:22<10:34,  7.38s/bat
Processing batches in day_2

Processing batches in day_28_description: 100%|█| 97/97 [12:24<00:00,  7.68s/bat
Processing columns: 100%|███████████████| 61/61 [13:01:54<00:00, 769.08s/column]


In [42]:
len(embeddings_dict)

61

In [43]:
output_file_path = os.path.join(output_dir, f"all_embeddings.npy")
np.save(output_file_path, embeddings_dict)

In [45]:
# embeddings_dict

In [46]:
# Compute the cosine similarity matrix
similarity_matrix = cosine_similarity(embeddings)


In [47]:
similarity_matrix

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)